# Santander Customer Transaction Prediction

# Importing libraries:

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from datetime import datetime
from scipy import stats
from sklearn.metrics import accuracy_score

In [2]:
#Upgrading XGBoost
!pip install --upgrade xgboost

    100% |████████████████████████████████| 114.0MB 297kB/s 
  Found existing installation: xgboost 0.7.post4
    Uninstalling xgboost-0.7.post4:
      Successfully uninstalled xgboost-0.7.post4


# Loading and checking the data

# Cleaning the data for the training step

In [3]:
df_train = pd.read_csv('train.csv')
df_train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df_test =pd.read_csv('test.csv')
df_test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [4]:
train_columns = df_train.drop(['ID_code','target'],axis =1).columns
train_columns

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

# Creating the model with XGBoost

In [0]:
import xgboost as xgb
d_train = xgb.DMatrix(df_train[train_columns],label = df_train['target'])
d_test = xgb.DMatrix(df_test[train_columns])

# Parameters

In [0]:
param_grid = {'max_depth':5,
                   'silent':1,
                   'eta':0.1, # Learning Rate
                   'gamma':0,
                   'min_child_weight':0.2784483175645849,
                   'objective':'binary:logistic',
                  }

In [0]:
xgbmodel = xgb.train(param_grid, d_train,500)
preds = xgbmodel.predict(d_test)

In [0]:
cv_results = xgb.cv(d_train())

# Predictions

In [12]:
predictions = df_test[['ID_code']]
predictions['target']=preds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
predictions.head()

,ID_code,target
0,test_0,0.060731
1,test_1,0.234987
2,test_2,0.136124
3,test_3,0.211610
4,test_4,0.047523


In [0]:
# saving results
predictions.to_csv("submissions.csv", index=False)

**Use this line to download your files to your computer:**

In [0]:
from google.colab import files
files.download('submissions.csv') 

In [17]:
submission = pd.read_csv('submissions.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
ID_code    200000 non-null object
target     200000 non-null float64
dtypes: float64(1), object(1)
memory usage: 3.1+ MB
